In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-securi

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-07-04 00:07:23--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.66MB/s    in 0.2s    

2021-07-04 00:07:23 (5.66 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
# May 2018 citibike data
from pyspark import SparkFiles
url = "https://bootcamp-citibike-bucket.s3.amazonaws.com/NYC/May2018.csv"
spark.sparkContext.addFile(url)
df2018 = spark.read.options(header=True, inferSchema=True).csv(SparkFiles.get("May2018.csv"))
df2018.show()

+------------+--------------------+--------------------+----------------+------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+------+----------+----------+------+
|tripduration|           starttime|            stoptime|start station id|start station name|start station latitude|start station longitude|end station id|    end station name|end station latitude|end station longitude|bikeid|  usertype|birth year|gender|
+------------+--------------------+--------------------+----------------+------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+------+----------+----------+------+
|         367|2018-05-01 05:06:...|2018-05-01 05:12:...|              72|  W 52 St & 11 Ave|           40.76727216|           -73.99392888|           514|    12 Ave & W 40 St|         40.76087502|         -74.00277668| 30567|Subscriber

In [5]:
#May 2019 citibike data
from pyspark import SparkFiles
url = "https://bootcamp-citibike-bucket.s3.amazonaws.com/NYC/May2019.csv"
spark.sparkContext.addFile(url)
df2019 = spark.read.options(header=True, inferSchema=True).csv(SparkFiles.get("May2019.csv"))
df2019.show()

+------------+--------------------+--------------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+------+----------+----------+------+
|tripduration|           starttime|            stoptime|start station id|  start station name|start station latitude|start station longitude|end station id|    end station name|end station latitude|end station longitude|bikeid|  usertype|birth year|gender|
+------------+--------------------+--------------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+------+----------+----------+------+
|         139|2019-05-01 00:00:...|2019-05-01 00:02:...|             447|     8 Ave & W 52 St|           40.76370739|            -73.9851615|           423|     W 54 St & 9 Ave|         40.76584941|         -73.98690506| 31170|Su

In [6]:
# May 2020 citibike data
from pyspark import SparkFiles
url = "https://bootcamp-citibike-bucket.s3.amazonaws.com/NYC/May2020.csv"
spark.sparkContext.addFile(url)
df2020 = spark.read.options(header=True, inferSchema=True).csv(SparkFiles.get("May2020.csv"))
df2020.show()

+------------+--------------------+--------------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+------+----------+----------+------+
|tripduration|           starttime|            stoptime|start station id|  start station name|start station latitude|start station longitude|end station id|    end station name|end station latitude|end station longitude|bikeid|  usertype|birth year|gender|
+------------+--------------------+--------------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+------+----------+----------+------+
|         336|2020-05-01 00:01:...|2020-05-01 00:06:...|            2017|     E 43 St & 2 Ave|           40.75022392|           -73.97121414|           465|  Broadway & W 41 St|         40.75513557|         -73.98658032| 18015|Su

In [7]:
# May 2021 citibike data
from pyspark import SparkFiles
url = "https://bootcamp-citibike-bucket.s3.amazonaws.com/NYC/May2021.csv"
spark.sparkContext.addFile(url)
df2021 = spark.read.options(header=True, inferSchema=True).csv(SparkFiles.get("May2021.csv"))
df2021.show()

+----------------+-------------+-------------------+-------------------+--------------------+----------------+--------------------+--------------+---------+----------+------------------+------------------+-------------+
|         ride_id|rideable_type|         started_at|           ended_at|  start_station_name|start_station_id|    end_station_name|end_station_id|start_lat| start_lng|           end_lat|           end_lng|member_casual|
+----------------+-------------+-------------------+-------------------+--------------------+----------------+--------------------+--------------+---------+----------+------------------+------------------+-------------+
|E12103A91226099A|  docked_bike|2021-05-06 17:25:01|2021-05-06 17:45:25|  Broadway & W 60 St|         6948.11|     1 Ave & E 78 St|       7020.09|40.769155|-73.981918|       40.77140426|       -73.9535166|       member|
|26A3DC47FE0EA3A3|  docked_bike|2021-05-13 12:48:08|2021-05-13 13:07:37|  Broadway & W 25 St|         6173.08|   E 2 St 

In [8]:
# May 2018 station data

from pyspark import SparkFiles
url = "https://bootcamp-citibike-bucket.s3.amazonaws.com/StationDocks/StationsRawMay2018.csv"
spark.sparkContext.addFile(url)
stations2018 = spark.read.options(delimiter = "\t", header=True, inferSchema=True).csv(SparkFiles.get("StationsRawMay2018.csv"))
stations2018.show()

+-------+----------------+--------+----+------+---+-----------+-----------+---------+---------+----------+----------+----------+
|dock_id|       dock_name|    date|hour|minute| pm|avail_bikes|avail_docks|tot_docks|     _lat|     _long|in_service|status_key|
+-------+----------------+--------+----+------+---+-----------+-----------+---------+---------+----------+----------+----------+
|     72|W 52 St & 11 Ave|18-05-01|   1|     4|  0|         16|         20|       39|40.767272|-73.993929|         1|         1|
|     72|W 52 St & 11 Ave|18-05-01|   1|    36|  0|         16|         20|       39|40.767272|-73.993929|         1|         1|
|     72|W 52 St & 11 Ave|18-05-01|   2|     7|  0|         16|         20|       39|40.767272|-73.993929|         1|         1|
|     72|W 52 St & 11 Ave|18-05-01|   2|    38|  0|         16|         20|       39|40.767272|-73.993929|         1|         1|
|     72|W 52 St & 11 Ave|18-05-01|   3|     9|  0|         16|         20|       39|40.767272|-7

In [9]:
# April 2019 station data

from pyspark import SparkFiles
url = "https://bootcamp-citibike-bucket.s3.amazonaws.com/StationDocks/StationsRawApril2019.csv"
spark.sparkContext.addFile(url)
stations2019 = spark.read.options(delimiter = "\t", header=True, inferSchema=True).csv(SparkFiles.get("StationsRawApril2019.csv"))
stations2019.show()

+-------+----------------+--------+----+------+---+-----------+-----------+---------+---------+----------+----------+----------+
|dock_id|       dock_name|    date|hour|minute| pm|avail_bikes|avail_docks|tot_docks|     _lat|     _long|in_service|status_key|
+-------+----------------+--------+----+------+---+-----------+-----------+---------+---------+----------+----------+----------+
|     72|W 52 St & 11 Ave|19-04-02|   4|    56|  1|          6|         49|       55|40.767272|-73.993929|         1|         1|
|     72|W 52 St & 11 Ave|19-04-02|   5|    36|  1|          1|         54|       55|40.767272|-73.993929|         1|         1|
|     72|W 52 St & 11 Ave|19-04-02|   6|    16|  1|          4|         51|       55|40.767272|-73.993929|         1|         1|
|     72|W 52 St & 11 Ave|19-04-02|   6|    54|  1|          3|         52|       55|40.767272|-73.993929|         1|         1|
|     72|W 52 St & 11 Ave|19-04-02|   7|    36|  1|          7|         48|       55|40.767272|-7

In [10]:
stations2019.dtypes

[('dock_id', 'int'),
 ('dock_name', 'string'),
 ('date', 'string'),
 ('hour', 'int'),
 ('minute', 'int'),
 ('pm', 'int'),
 ('avail_bikes', 'int'),
 ('avail_docks', 'int'),
 ('tot_docks', 'int'),
 ('_lat', 'double'),
 ('_long', 'double'),
 ('in_service', 'int'),
 ('status_key', 'int')]

In [11]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://citibike-analysis.cbk1sw5fz5td.us-east-1.rds.amazonaws.com:5432/project"
config = {"user":"postgres", 
          "password": "postgres", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write may2018 to table in RDS
df2018.write.jdbc(url=jdbc_url, table='may2018', mode=mode, properties=config)

In [12]:
# Write may2019 to table in RDS
df2019.write.jdbc(url=jdbc_url, table='may2019', mode=mode, properties=config)

In [13]:
# Write may2020 to table in RDS
df2020.write.jdbc(url=jdbc_url, table='may2020', mode=mode, properties=config)

In [17]:
# Write may2021 to table in RDS
df2021.write.jdbc(url=jdbc_url, table='may2021', mode=mode, properties=config)

In [15]:
# Write stations2018 to table in RDS
stations2018.write.jdbc(url=jdbc_url, table='stations2018', mode=mode, properties=config)

In [16]:
# Write stations2019 to table in RDS
stations2019.write.jdbc(url=jdbc_url, table='stations2019', mode=mode, properties=config)